In [9]:
import requests
import pandas as pd
import plotly.graph_objects as go
from statsmodels.tsa.statespace.sarimax import SARIMAX
 
# --- Parámetros ---
token     = "45b0d369-41fd-467a-b7c9-6b862cf3056d"      # <— reemplaza
indicator = "910493"              # INPC general 444558 O 910424
geo       = "00000"               # Nacional (5 ceros)
fuente    = "BIE"                 # Banco de Información Económica (prueba BISE si hiciera falta)
url = (
    "https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/"
    f"INDICATOR/{indicator}/es/{geo}/false/{fuente}/2.0/{token}?type=json"
)
 
# --- Llamada API con manejo de errores legible ---
resp = requests.get(url)
if resp.status_code != 200:
    raise SystemExit(f"INEGI {resp.status_code}: {resp.text[:400]}")
 
j = resp.json()

In [12]:
# --- Graficar serie de tiempo (a partir del JSON `j`) ---

import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "browser"   # abre en navegador por defecto


# 1) Convertir JSON a DataFrame mensual
series_key = "Series" if "Series" in j else ("series" if "series" in j else None)
obs = j[series_key][0]["OBSERVATIONS"]

df = (
    pd.DataFrame(obs)[["TIME_PERIOD", "OBS_VALUE"]]
    .rename(columns={"TIME_PERIOD": "fecha", "OBS_VALUE": "valor"})
)
df["fecha"] = pd.to_datetime(df["fecha"], errors="coerce")
df["valor"] = pd.to_numeric(df["valor"], errors="coerce")
df = df.dropna().sort_values("fecha").set_index("fecha")
# asegurar frecuencia mensual (por si viniera irregular)
df = df.asfreq("MS") if df.index.inferred_freq is None else df
df = df.resample("MS").last()

# 2) Gráfica interactiva con Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df.index, y=df["valor"],
    mode="lines",
    name="Serie observada"
))
fig.update_layout(
    title=f"Serie de tiempo — Indicador {indicator}",
    xaxis_title="Fecha",
    yaxis_title="Índice",
    hovermode="x unified",
    template="plotly_white"
)
fig.show()


/var/folders/fd/q5kbvbvd00b4rdf_lfzx0j3m0000gp/T/ipykernel_69593/3599741156.py:17: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



In [13]:
import plotly.express as px

# Filtrar desde 2015
df_2015 = df[df.index >= "2015-01-01"].copy()

# Crear columnas auxiliares para separar por año y mes
df_2015["Año"] = df_2015.index.year
df_2015["Mes"] = df_2015.index.month

# Gráfica de líneas: cada año una curva (mes en eje X)
fig = px.line(
    df_2015,
    x="Mes", y="valor",
    color="Año",
    markers=True,
    title="Comportamiento del INPC por año (2015 en adelante)"
)

fig.update_layout(
    xaxis=dict(tickmode="array", tickvals=list(range(1, 13))),
    yaxis_title="INPC",
    xaxis_title="Mes",
    legend_title="Año",
    template="plotly_white"
)

fig.show()
